<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#Q1">Question 1</a></li>
<li><a href='#Q2'>Question 2</a></li>
<li><a href='#Q3'>Question 3</a></li>
<li><a href="#Q4">Question 4</a></li>
<li><a href='#Q5'>Question 5</a></li>
<li><a href='#Q6'>Question 6</a></li>
<li><a href="#Q7">Question 7</a></li>
<li><a href='#Q8'>Question 8</a></li>
<li><a href='#Q9'>Question 9</a></li>
<li><a href='#bonus1'>Bonus Question 1</a></li>
<li><a href='#bonus2'>Bonus Question 2</a></li>

Do you work for this exercise in either a jupyter notebook named evaluate within your regression-exercises repository. By the end of this exercise you will also create a python script named evaluate, so the overall deliverables for this exercise are the python script and jupyter notebook both with the name evaluate within your regression-exercises repo.

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [5]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats
from scipy import stats

# Datasets
from pydataset import data

# .py files
import wrangle

<a id='Q1'></a>
# Question 1:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 1. Load your zillow dataset.

In [4]:
zillow = wrangle.wrangle_zillow()
zillow.sample()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
772036,3,2.0,1234.0,236043.0,1952,2837.08,6059


<a id='Q2'></a>
# Question 2:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 2. Fit a linear regression model (ordinary least squares) and compute yhat, predictions of taxvaluedollarcnt using only calculatedfinishedsqft.

Here is some sample code to get you started:

from sklearn.linear_model import LinearRegression

assuming X and y are already defined
model = LinearRegression().fit(X, y)
predictions = model.predict(X)

Modify and add to the code above as necessary for it to work with the zillow dataset.

<a id='Q3'></a>
# Question 3:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 3. Plot the residuals for the linear regression model that you made.

<a id='Q4'></a>
# Question 4:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 4. Calculate the sum of squared errors, explained sum of squares, total sum of squares, mean squared error, and root mean squared error for your model.

<a id='Q5'></a>
# Question 5:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 5. Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average taxvaluedollarcnt amount).

<a id='Q6'></a>
# Question 6:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 6. Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.

<a id='Q7'></a>
# Question 7:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 7. What is the amount of variance explained in your model?

<a id='Q8'></a>
# Question 8:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 8. Is your model better than the baseline model?

<a id='Q9'></a>
# Question 9:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 9 . Create a file named evaluate.py that contains the following functions.

##### 9a. plot_residuals(y, yhat): creates a residual plot

##### 9b. regression_errors(y, yhat): returns the following values:
- sum of squared errors (SSE)
- explained sum of squares (ESS)
- total sum of squares (TSS)
- mean squared error (MSE)
- root mean squared error (RMSE)

##### 9c. baseline_mean_errors(y): computes the SSE, MSE, and RMSE for the baseline model

##### 9d. better_than_baseline(y, yhat): returns true if your model performs better than the baseline, otherwise false

<a id='bonus1'></a>
# Bonus Question 1:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### Bonus - 1. Load the tips dataset from either pydataset or seaborn. Fit a linear regression model (ordinary least squares) and compute yhat, predictions of tip using total_bill.

In [7]:
tips = data('tips')
tips.sample()

,total_bill,tip,sex,smoker,day,time,size
219,7.74,1.44,Male,Yes,Sat,Dinner,2


<a id='bonus2'></a>
# Bonus Question 2:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### Bonus - 2. Load the mpg dataset and fit a model that predicts highway mileage based on engine displacement. Take a look at all the regression evaluation metrics, and determine whether this model is better than the baseline model. Use the functions from your evaluate.py to help accomplish this.

In [9]:
mpg = data('mpg')
mpg.sample()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
130,jeep,grand cherokee 4wd,6.1,2008,8,auto(l5),4,11,14,p,suv
